In [1]:
from GeneralTools.misc_fun import FLAGS

FLAGS.TENSORFLOW_VERSION


'1.13.0'

In [9]:
import tensorflow as tf
"""
Scope - continue

tf.variable_scope vs tf.name_scope
 -- variable_scope applies to constants, variables and operations
 -- name_scope applies to constants and operations, but not variables
"""

with tf.Graph().as_default():
    a1 = tf.constant(0.1, name='const_a1')
    b1 = tf.get_variable(
        name='var_b1', dtype=tf.float32, 
        initializer=a1)
    c1 = tf.multiply(a1, b1, name='op_1')

    with tf.variable_scope('layer01'):
        # variable scope applies to constants, variables and operations
        a2 = tf.constant(0.1, name='const_a2')
        b2 = tf.get_variable(
            name='var_b2', dtype=tf.float32, 
            initializer=a1)  # initializer could be a1, b1, a2
        c2 = tf.multiply(a2, b2, name='op_2')
        
    with tf.name_scope('layer02'):
        # name scope applies to constants and operations
        a3 = tf.constant(0.1, name='const_a3')
        # name scope does not apply to variables
        b3 = tf.get_variable(
            name='var_b3', dtype=tf.float32, 
            initializer=a1)  # initialize could be a1, b1, a2, b2, a3
        c3 = tf.multiply(a3, b3, name='op_3')
    
print("The name/operation of a1 is '{}'".format(a1.name))
print("The name/operation of b1 is '{}'".format(b1.name))
print("The name/operation of c1 is '{}'".format(c1.name))
print('------')
print("The name/operation of a2 is '{}'".format(a2.name))
print("The name/operation of b2 is '{}'".format(b2.name))
print("The name/operation of c2 is '{}'".format(c2.name))
print('------')
print("The name/operation of a3 is '{}'".format(a3.name))
print("The name/operation of b3 is '{}'".format(b3.name))
print("The name/operation of c3 is '{}'".format(c3.name))


The name/operation of a1 is 'const_a1:0'
The name/operation of b1 is 'var_b1:0'
The name/operation of c1 is 'op_1:0'
------
The name/operation of a2 is 'layer01/const_a2:0'
The name/operation of b2 is 'layer01/var_b2:0'
The name/operation of c2 is 'layer01/op_2:0'
------
The name/operation of a3 is 'layer02/const_a3:0'
The name/operation of b3 is 'var_b3:0'
The name/operation of c3 is 'layer02/op_3:0'


In [2]:
import tensorflow as tf
from GeneralTools.my_input import DatasetFromTensor, create_placeholder
"""
Six-step workflow of data analysis:
1. prepare data
2. define the model
3. define the loss function
4. configure the optimizer
5. configure the summary
6. call a session
"""

with tf.Graph().as_default():
    # load the data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    data_tr = DatasetFromTensor(
        (x_train, y_train), (tf.float32, tf.int32), 
        map_func=lambda x, y: (x/255.0, y),
        name='TrainData').schedule(400)
    xtr, ytr = data_tr.next()
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        yv = sess.run(y)
        print(yv.shape)


(tf.float32, tf.int32)
(TensorShape([Dimension(28), Dimension(28)]), TensorShape([]))
Instructions for updating:
Colocations handled automatically by placer.


Tensor("IteratorGetNext:0", shape=(?, 28, 28), dtype=float32)
Tensor("IteratorGetNext:1", shape=(?,), dtype=int32)


In [1]:
import tensorflow as tf
from GeneralTools.my_model import SequentialNet

with tf.Graph().as_default():
    x = tf.random_normal((64, 28, 28))
    
    net_design = [{'name': 'l1', 'out': 33, 'op': 'c', 'in_reshape': (1, 28, 28)}, 
                  {'name': 'l2', 'out': 66, 'op': 'c', 'stride': 2}, 
                  {'name': 'l3', 'out': 11, 'op': 'c', 'stride': 2}]
    mdl = SequentialNet(net_design, input_shape=[None, 28, 28], name='Conv')
    
    y = mdl(x)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        yv = sess.run(y)
        print(yv.shape)


Instructions for updating:
Colocations handled automatically by placer.


(64, 11, 28, 28)


TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("random_normal:0", shape=(100, 3), dtype=float32) which was passed to the feed with key Tensor("Placeholder:0", shape=(100, 3), dtype=float32).